In [1]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [3]:
### COMPLETE WORK OF SHAKESPEARE
### http://www.gutenberg.org/ebooks/100

data = sc.textFile("data/100-0.txt")

In [4]:
data.take(3)

['',
 'Project Gutenberg’s The Complete Works of William Shakespeare, by William',
 'Shakespeare']

In [9]:
data.collect()

['',
 'Project Gutenberg’s The Complete Works of William Shakespeare, by William',
 'Shakespeare',
 '',
 'This eBook is for the use of anyone anywhere in the United States and',
 'most other parts of the world at no cost and with almost no restrictions',
 'whatsoever.  You may copy it, give it away or re-use it under the terms',
 'of the Project Gutenberg License included with this eBook or online at',
 'www.gutenberg.org.  If you are not located in the United States, you’ll',
 'have to check the laws of the country where you are located before using',
 'this ebook.',
 '',
 '',
 'Title: The Complete Works of William Shakespeare',
 '',
 'Author: William Shakespeare',
 '',
 'Release Date: January 1994 [EBook #100]',
 'Last Updated: November 7, 2019',
 '',
 'Language: English',
 '',
 'Character set encoding: UTF-8',
 '',
 '*** START OF THIS PROJECT GUTENBERG EBOOK THE COMPLETE WORKS OF WILLIAM SHAKESPEARE ***',
 '',
 '',
 '',
 '',
 'The Complete Works of William Shakespeare',
 '',
 '',
 '

In [10]:
data.first()

''

In [8]:
## number of lines
data.count() ### 166,902

166902

In [15]:
#TakeSample (withReplacement, n, [seed]) - This action will return n elements from the dataset, 
#with or without replacement (true or false). Seed is an optional parameter that is used as a random generator.

#data.takeSample(False, 10, 2)
data.takeSample(False, 10)


['  AUMERLE. Princes, and noble lords,',
 'PATROCLUS.',
 ' [_Exeunt._]',
 'Until this instant. Take a trumpet, herald;',
 'But under heavy judgement bears that life',
 '    But cheerly seek how to redress their harms.',
 '',
 '  CHARLES. Somewhat too sudden, sirs, the warning is;',
 'QUINCE.',
 'Till then I’ll keep him dark, and safely lock’d.']

In [16]:
#divide each string on spaces
mappeddata = data.map(lambda line : line.split(" "))
mappeddata.take(5)

[[''],
 ['Project',
  'Gutenberg’s',
  'The',
  'Complete',
  'Works',
  'of',
  'William',
  'Shakespeare,',
  'by',
  'William'],
 ['Shakespeare'],
 [''],
 ['This',
  'eBook',
  'is',
  'for',
  'the',
  'use',
  'of',
  'anyone',
  'anywhere',
  'in',
  'the',
  'United',
  'States',
  'and']]

In [17]:
flatMappeddata = data.flatMap(lambda line : line.split(" "))
flatMappeddata.take(5)

['', 'Project', 'Gutenberg’s', 'The', 'Complete']

In [24]:
##Let’s try to find out the lines having confusion term in it in data RDD

#onlyconfusion = data.filter(lambda line : ("confusion" in line.lower()))
onlyconfusion = data.filter(lambda line : "confusion" in line)
onlyconfusion.count()

34

In [25]:
# to find out what are those lines we can use the collect() action in Spark as shown below 
onlyconfusion.collect()

['    To our confusion.',
 '  HYMEN.    Peace, ho! I bar confusion;',
 'Infect thy sap, and live on thy confusion.',
 '    Neither supreme, how soon confusion',
 '    Crying confusion.',
 'Receive it from me, then: war and confusion',
 'Nay, to thy mere confusion, thou shalt know',
 'A rout, confusion thick. Forthwith they fly,',
 'Get from him why he puts on this confusion,',
 'Of pellmell havoc and confusion.',
 '    You may behold confusion of your foes.                  Exit',
 '    There comes the ruin, there begins confusion.           Exit',
 '    Heaping confusion on their own heads thereby!',
 '  YOUNG CLIFFORD. Shame and confusion! All is on the rout;',
 '    Then let confusion of one part confirm',
 '    Meet in one line; and vast confusion waits,',
 ' [_Dies. The Senators and People retire in confusion._]',
 'Vengeance! plague! death! confusion!',
 '   Come to great confusion:',
 'Shall draw him on to his confusion.',
 'than sand-blind, high-gravel blind, knows me not. I wi

In [28]:
sampledconfusion = data.sample(True,0.5,3) #True implies withReplacement
                                           # The fraction means % of total data you want to take the sample from (50% here)
sampledconfusion.collect()
sampledconfusion.count()


83527

In [29]:
### UNION EXAMPLE

abhay_marks = [("physics",85),("maths",75),("chemistry",95)]
ankur_marks = [("physics",65),("maths",45),("chemistry",85)]

abhay = sc.parallelize(abhay_marks)
ankur = sc.parallelize(ankur_marks)

abhay.union(ankur).collect()

[('physics', 85),
 ('maths', 75),
 ('chemistry', 95),
 ('physics', 65),
 ('maths', 45),
 ('chemistry', 85)]

In [30]:
### JOIN EXAMPLE
#joins two RDDs based on a common key, you can combine the marks of Abhay and Ankur based on each subject as follows
Subject_wise_marks = abhay.join(ankur)
Subject_wise_marks.collect()

[('physics', (85, 65)), ('maths', (75, 45)), ('chemistry', (95, 85))]

In [31]:
### INTERSECTION EXAMPLE
#common terms or objects from the two RDDS.
#example: Let’s find out the players who are both good cricketers as well as toppers of the class.

Cricket_team = ["sachin","abhay","michael","rahane","david","ross","raj","rahul","hussy","steven","sourav"]
Toppers = ["rahul","abhay","laxman","bill","steve"]

cricketRDD = sc.parallelize(Cricket_team)
toppersRDD = sc.parallelize(Toppers)

toppercricketers = cricketRDD.intersection(toppersRDD)
toppercricketers.collect()

['rahul', 'abhay']

In [32]:
### DISTINCT EXAMPLE
#get rid of any ambiguities. As the name suggest it picks out the lines from the RDD that are unique.
#Example: Suppose that there are various movie nominations in different categories. 
#We want to find out, how many movies are nominated overall

best_story = ["movie1","movie3","movie7","movie5","movie8"]
best_direction = ["movie11","movie1","movie5","movie10","movie7"]
best_screenplay = ["movie10","movie4","movie6","movie7","movie3"]

story_rdd = sc.parallelize(best_story)
direction_rdd = sc.parallelize(best_direction)
screen_rdd = sc.parallelize(best_screenplay)

total_nomination_rdd = story_rdd.union(direction_rdd).union(screen_rdd)
total_nomination_rdd.collect()

['movie1',
 'movie3',
 'movie7',
 'movie5',
 'movie8',
 'movie11',
 'movie1',
 'movie5',
 'movie10',
 'movie7',
 'movie10',
 'movie4',
 'movie6',
 'movie7',
 'movie3']

In [34]:
unique_movies_rdd = total_nomination_rdd.distinct()
unique_movies_rdd.collect()

['movie10',
 'movie8',
 'movie5',
 'movie1',
 'movie4',
 'movie7',
 'movie3',
 'movie11',
 'movie6']

In [35]:
##### RDDs
data.getNumPartitions()

2

In [36]:
# A helper function to compute the list of words in a line of text
import re
def get_words(line):
    return re.compile('\w+').findall(line)

print(get_words("This, is a test!"))

['This', 'is', 'a', 'test']


In [39]:
import re
pattern=re.compile('\w+')#'TP')
result=pattern.findall('TP Tutorialspoint TP')
print(result)
result2=pattern.findall('TP is most popular tutorials site of India')
print(result2)

['TP', 'Tutorialspoint', 'TP']
['TP', 'is', 'most', 'popular', 'tutorials', 'site', 'of', 'India']


In [44]:
warcount = data.filter(lambda line: "war" in get_words(line)).count()
print(warcount)
peacecount = data.filter(lambda line: "peace" in get_words(line)).count()
print(peacecount)

277
401


In [42]:
for line in data.filter(lambda line: "war" in get_words(line)).take(5):
    print(line)

Sweets with sweets war not, joy delights in joy:
  And all in war with Time for love of you,
Make war upon this bloody tyrant Time?
Such civil war is in my love and hate,
Mine eye and heart are at a mortal war,


In [47]:
### Use flatMap() to create an RDD of the words and count the number of words.

# Split each line into words using get_words()
words = data.flatMap(lambda line: get_words(line))
words.take(5)

['Project', 'Gutenberg', 's', 'The', 'Complete']

In [46]:
flatMappeddata = data.flatMap(lambda line : line.split(" "))
flatMappeddata.take(5)

['', 'Project', 'Gutenberg’s', 'The', 'Complete']

In [48]:
words.count()

993836

In [49]:
words.distinct().count()

33058

In [50]:
# Split each line into words using get_words() 
words = data.map(lambda line: get_words(line)) ### used map and not flatmap!
words.take(5)

[[],
 ['Project',
  'Gutenberg',
  's',
  'The',
  'Complete',
  'Works',
  'of',
  'William',
  'Shakespeare',
  'by',
  'William'],
 ['Shakespeare'],
 [],
 ['This',
  'eBook',
  'is',
  'for',
  'the',
  'use',
  'of',
  'anyone',
  'anywhere',
  'in',
  'the',
  'United',
  'States',
  'and']]

In [55]:
#Use the function union() to create an RDD of lines with either war or peace mentioned. Count how many lines.
warLines = data.filter(lambda line: "war" in get_words(line))
peaceLines = data.filter(lambda line: "peace" in get_words(line))
warOrPeaceLines = peaceLines.union(warLines)
warOrPeaceLines.count()
warOrPeaceLines.take(5)

['And for the peace of you I hold such strife',
 'And peace proclaims olives of endless age.',
 'Bless him at home in peace, whilst I from far',
 'I hear there is an overture of peace.',
 'Nay, I assure you, a peace concluded.']

In [56]:
warOrPeaceLines.collect()

['And for the peace of you I hold such strife',
 'And peace proclaims olives of endless age.',
 'Bless him at home in peace, whilst I from far',
 'I hear there is an overture of peace.',
 'Nay, I assure you, a peace concluded.',
 '    Thy soldier, servant, making peace or war',
 '    Which fronted mine own peace. As for my wife,',
 '    And though I make this marriage for my peace,',
 '    Shall make thy peace for moving me to rage;',
 '    And make your peace with Caesar.',
 '  CLEOPATRA. Prithee, peace.',
 '  CAESAR. The time of universal peace is near.',
 '  ANTONY. Hence, saucy eunuch; peace!',
 '  CHARMIAN. Peace, peace, Iras!',
 '  CLEOPATRA. Peace, peace!',
 '    brothers. Your If is the only peace-maker; much virtue in If.',
 'Peace, doting wizard, peace; I am not mad.',
 '    That like nor peace nor war? The one affrights you,',
 '    the peace you make in their cause is calling both the parties',
 '  PATRICIANS. Peace, peace, peace; stay, hold, peace!',
 '  SICINIUS. Hear me,

In [57]:
warAndPeaceLines = warLines.intersection(peaceLines)
warAndPeaceLines.count()

19

In [58]:
warAndPeaceLines.collect()

['    Thy soldier, servant, making peace or war',
 '    As likely to be blest in peace and war;',
 'A moth of peace, and he go to the war,',
 '    Your discipline in war, wisdom in peace,',
 'To offer war where they should kneel for peace,',
 'How he in peace is wounded, not in war.',
 '    That like nor peace nor war? The one affrights you,',
 '  FIRST SERVANT. Let me have war, say I; it exceeds peace as far as',
 '    That war, or peace, or both at once, may be',
 '  PLANTAGENET. And peace, no war, befall thy parting soul!',
 '  YORK. Thus war hath given thee peace, for thou art still.',
 '    Make war breed peace, make peace stint war, make each',
 '    To speak of peace or war.',
 '    One that still motions war and never peace,',
 '  CONSTANCE. War! war! no peace! Peace is to me a war.',
 '    And prosperous be thy life in peace and war!          [Dies]',
 '    Not mutinous in peace, yet bold in war;',
 '    That right in peace which here we urge in war,',
 '    Would they make pe

In [59]:
a = data.filter(lambda line: "war" in get_words(line) and "peace" in get_words(line))#.take(5)
a.count()

19

In [60]:
a.take(1)

['    Thy soldier, servant, making peace or war']

In [61]:
#You have already seen three actions: collect() which returns all elements in the RDD, take(n), 
#which return the first n elements of the RDD, and count() which returns the number of elements in the RDD.
#The action reduce() takes as input a function which collapses two elements into one. Use it to find the 
#longest word 

data.flatMap(lambda line: get_words(line)).reduce(lambda acc, line: line if len(line) > len(acc) else acc)

'_______________________________________________________________'

In [63]:
#find the set of words which only appear titlecased.

words = data.flatMap(lambda line: get_words(line)).distinct()
titled_words = words.filter(lambda word: word.istitle()) # maybe there is a "But", "It" => titlecased because begins a sentence
print(titled_words.take(5))#count())

['Project', 'The', 'Shakespeare', 'United', 'States']


In [64]:
untitled_words = words.filter(lambda word: not word.istitle()) ### all lowercase ones and UPPERCASE ones
print(untitled_words.take(5))#count())

['s', 'of', 'is', 'use', 'anyone']


In [65]:
lower_titles = titled_words.map(lambda word: word.lower()) ## changed to "but", "it" ...
print(lower_titles.take(5))#count())

['project', 'the', 'shakespeare', 'united', 'states']


In [66]:
nouns = lower_titles.subtract(untitled_words) ### removed the "but", "it" ...!
nouns.take(50)

['unmoved',
 'slandering',
 'dramatis',
 'helena',
 'narbon',
 'tom',
 'jaques',
 'bajazet',
 'fontybell',
 'lodowick',
 'tartar',
 'lavache',
 'maudlin',
 '_bertram',
 'necessitied',
 'parolles_',
 'herod',
 'nilus',
 'egyptian',
 'pompeius',
 'messina',
 'maecenas',
 'orodes',
 'parthians',
 'sossius',
 'caesarion',
 'thracian',
 'actium',
 'ingross',
 'submits',
 'luxuriously',
 'casaer',
 'condemning',
 'immoment',
 'livia',
 'frederick',
 'arden',
 'celia',
 'envenoms',
 'phebe',
 'rosalinda',
 'goths',
 'goddild',
 'hellespont',
 'saturdays',
 'portugal',
 'audre',
 'egeon',
 'excludes',
 'centaur']

In [67]:
############# Key/Value pairs in Spark

words = data.flatMap(lambda line: get_words(line))
words.take(5)
word_pairs = words.map(lambda word: (word, 1))
word_pairs.take(5)

[('Project', 1), ('Gutenberg', 1), ('s', 1), ('The', 1), ('Complete', 1)]

In [68]:
from operator import add
###rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 1)])
sorted(word_pairs.reduceByKey(add).collect())

[('0', 3),
 ('000', 1),
 ('1', 120),
 ('10', 3),
 ('100', 6),
 ('1000', 1),
 ('1004', 1),
 ('1009', 1),
 ('101', 1),
 ('1012', 1),
 ('1016', 1),
 ('102', 1),
 ('1020', 1),
 ('1024', 1),
 ('1028', 1),
 ('103', 1),
 ('1033', 1),
 ('1036', 1),
 ('104', 2),
 ('1040', 1),
 ('1044', 1),
 ('1049', 1),
 ('105', 1),
 ('1053', 1),
 ('1057', 1),
 ('106', 1),
 ('1060', 1),
 ('1065', 1),
 ('1069', 1),
 ('107', 1),
 ('1072', 1),
 ('1078', 1),
 ('108', 2),
 ('1081', 1),
 ('1085', 1),
 ('1088', 1),
 ('109', 1),
 ('1093', 1),
 ('1096', 1),
 ('11', 1),
 ('110', 1),
 ('1100', 1),
 ('1105', 1),
 ('1108', 1),
 ('111', 1),
 ('1112', 1),
 ('1116', 1),
 ('112', 2),
 ('1120', 1),
 ('1124', 1),
 ('1129', 1),
 ('113', 1),
 ('1132', 1),
 ('1136', 1),
 ('114', 1),
 ('1141', 1),
 ('1144', 1),
 ('1148', 1),
 ('115', 1),
 ('1152', 1),
 ('1156', 1),
 ('116', 2),
 ('1160', 1),
 ('1164', 1),
 ('1168', 1),
 ('117', 1),
 ('1172', 1),
 ('118', 1),
 ('1180', 1),
 ('1184', 1),
 ('1189', 1),
 ('119', 1),
 ('1192', 1),
 ('12',